### Import des bibliothéques
---

In [1]:
import pandas as pd
import requests as rq
import json as js

### Chargement des données
---

In [2]:
df_ville = pd.read_csv('df_ville.csv')
df_ville.head()

,nom_ville,latitude,longitude,id_ville
0,Mont Saint Michel,48.635954,-1.511460,1
1,St Malo,48.649518,-2.026041,2
2,Bayeux,49.276462,-0.702474,3
3,Le Havre,49.493898,0.107973,4
4,Rouen,49.440459,1.093966,5


### Scraping méteo Site openweathermap
---
#### prévisions météorologiques à 5 jours - données météorologiques par pas de 3 heures

In [3]:
# Site Openweathermap : https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&cnt=40&appid={API_KEY}&units=metric&lang=fr"
   
API_KEY = 'b9b15b3ef33eb08f185286b5682ad89d'
URL = 'https://api.openweathermap.org/data/2.5/forecast'


### Uniformiser les noms de colonnes 

In [4]:
# fonction nom de ville du df_ville identique au nom de ville API méteo Openweather ?

def check_city(city_name):
    """Retourne True si la ville est reconnue par l'API, False sinon"""
    params = {'q': f'{city_name},FR', 'appid': API_KEY}
    response = rq.get(
        URL, 
        params=params)
    
    data = response.json()
    if data.get('cod') == '200':
        return True
    else:
        return False

df_ville['ville_valide'] = df_ville['nom_ville'].apply(check_city)

print("\n--- Nombre de ville True or False ---\n")
print(df_ville['ville_valide'].value_counts())
print(f"\n{(~df_ville['ville_valide']).sum()} noms de ville ne sont pas corrects") #~ inverse le booléan : false

# création df des villes non valides
df_ville_false = df_ville[df_ville['ville_valide']==False]
df_ville_false.head(6)



--- Nombre de ville True or False ---

ville_valide
True     28
False     7
Name: count, dtype: int64

7 noms de ville ne sont pas corrects


,nom_ville,latitude,longitude,id_ville,ville_valide
0,Mont Saint Michel,48.635954,-1.511460,1,False
1,St Malo,48.649518,-2.026041,2,False
9,Chateau du Haut Koenigsbourg,48.249411,7.344320,10,False
17,Gorges du Verdon,43.749656,6.328562,18,False
18,Bormes les Mimosas,43.150697,6.341928,19,False
21,Aix en Provence,43.529842,5.447474,22,False


In [5]:
# Récupérer avec la lat et lon, les noms de ville valide de l'API méteo Openweather
villes_correctes = []

for i, row in df_ville.iterrows() :
    lat = row['latitude']
    lon = row['longitude']

    try:
        params = {'lat': lat, 'lon': lon, 'appid': API_KEY}

        response = rq.get(
            URL,
            params=params, 
            timeout=10 # si le site ne répond pas s'arrête ds les 10s
        )
        response.raise_for_status()
        data = response.json()

        Villes = data['city']['name']
        villes_correctes.append(Villes)

    except rq.RequestException as e:
        print(f"\tErreur pour {row['nom_ville']} : {e}")  
        villes_correctes.append(None)  # si erreur, on met None

# Ajouter la colonne d’un coup
df_ville['nom_ville'] = villes_correctes
df_ville=df_ville.drop(columns=['ville_valide'])
df_ville.head()

,nom_ville,latitude,longitude,id_ville
0,Huisnes-sur-Mer,48.635954,-1.511460,1
1,St-Malo,48.649518,-2.026041,2
2,Bayeux,49.276462,-0.702474,3
3,Le Havre,49.493898,0.107973,4
4,Rouen,49.440459,1.093966,5


### Scraper les infos méteo : température, 4 journées entiéres, tendance de la matinée et aprés-midi

In [6]:
import pandas as pd
import requests as rq
from datetime import datetime

API_KEY = 'b9b15b3ef33eb08f185286b5682ad89d'
URL = 'https://api.openweathermap.org/data/2.5/forecast'

# Création df_meteo 
df_meteo = pd.DataFrame(columns=['nom_ville','latitude','longitude','date','periode_journee','condition_meteo','temperature'])

# Boucle sur chaque ville
for i, row in df_ville.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    ville = row['nom_ville']  
    
    try:
        response = rq.get(
            URL,
            params={'lat': lat, 'lon': lon, 'appid': API_KEY, 'units': 'metric', 'lang': 'fr'},
            timeout=10
        )
        response.raise_for_status()
        data = response.json()
        
        if 'list' in data: 
            for forecast in data['list']:
                dt_txt = forecast['dt_txt']  # date et heure ss forme de texte ex: "2025-09-30 09:00:00"
                date_str, hour = dt_txt.split(" ") # on separe le schaines en 2 morceaux dat_str:"2025-09-30" hour : "09:00:00"
                date_format = datetime.strptime(date_str, "%Y-%m-%d").strftime("%Y-%m-%d") # transforme la date en format 30/09/2025
                
                if hour in ["09:00:00", "15:00:00"]:  # matin ou après-midi, 9h ou 15h, data ttes les 3h
                    temp = round(forecast['main']['temp'])
                    desc = forecast['weather'][0]['description']
                    periode = 'Matin' if hour=="09:00:00" else 'Après-midi'
                    
                    df_meteo = pd.concat([
                        df_meteo,
                        pd.DataFrame([{
                            'nom_ville': ville,
                            'latitude': lat,
                            'longitude': lon,
                            'date': date_format,
                            'periode_journee': periode,
                            'condition_meteo': desc,
                            'temperature': temp
                        }])
                    ], ignore_index=True)
        else:
            print(f"Aucune prévision pour {ville}")
    
    except rq.RequestException as e:
        print(f"Erreur pour {ville} : {e}")

# Vérification
df_meteo.head()


C:\Users\natha\AppData\Local\Temp\ipykernel_37044\1006948393.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_meteo = pd.concat([


,nom_ville,latitude,longitude,date,periode_journee,condition_meteo,temperature
0,Huisnes-sur-Mer,48.635954,-1.51146,2025-10-02,Après-midi,couvert,19
1,Huisnes-sur-Mer,48.635954,-1.51146,2025-10-03,Matin,couvert,17
2,Huisnes-sur-Mer,48.635954,-1.51146,2025-10-03,Après-midi,légère pluie,19
3,Huisnes-sur-Mer,48.635954,-1.51146,2025-10-04,Matin,pluie modérée,13
4,Huisnes-sur-Mer,48.635954,-1.51146,2025-10-04,Après-midi,légère pluie,15


### Ajout id_ville au df_meteo + id_meteo pour futurs tables SQL
---

In [9]:
df_meteo['condition_meteo'].value_counts()

condition_meteo
couvert                  125
ciel dégagé               84
nuageux                   48
légère pluie              38
partiellement nuageux     31
peu nuageux               18
pluie modérée              6
Name: count, dtype: int64

In [ ]:
df_ville['nom_ville'].value_counts()


nom_ville
Huisnes-sur-Mer                 1
Saintes-Maries-de-la-Mer        1
Marseille                       1
Aix-en-Provence                 1
Avignon                         1
Uzès                            1
Nîmes                           1
Aigues-Mortes                   1
Collioure                       1
Bormes-les-Mimosas              1
Carcassonne                     1
Larbont                         1
Toulouse                        1
Montauban                       1
Biarritz                        1
Le Limpou                       1
Cassis                          1
Bauduen                         1
St-Malo                         1
Arrondissement de Strasbourg    1
Bayeux                          1
Le Havre                        1
Rouen                           1
Paris                           1
Amiens                          1
Lille                           1
Saint-Hippolyte                 1
Vieux Lyon                      1
Colmar                          1
Egui

In [ ]:
# Merge pour avoir id_ville
df_meteo = df_meteo.merge(
    df_ville[['id_ville', 'nom_ville']], 
    on='nom_ville', 
    how='left'
)

# création id_meteo 
df_meteo['id_meteo'] = range(1, len(df_meteo)+1)

df_meteo.head()


,nom_ville,latitude,longitude,date,periode_journee,condition_meteo,temperature,id_ville,id_meteo
0,Huisnes-sur-Mer,48.635954,-1.51146,2025-10-01,Matin,couvert,15,1,1
1,Huisnes-sur-Mer,48.635954,-1.51146,2025-10-01,Après-midi,couvert,18,1,2
2,Huisnes-sur-Mer,48.635954,-1.51146,2025-10-02,Matin,couvert,16,1,3
3,Huisnes-sur-Mer,48.635954,-1.51146,2025-10-02,Après-midi,couvert,21,1,4
4,Huisnes-sur-Mer,48.635954,-1.51146,2025-10-03,Matin,légère pluie,17,1,5


### Sauvegarde df_meteo
---

In [ ]:
df_meteo.to_csv('df_meteo.csv', index=False, encoding='utf-8-sig')